# Introduction

This notebook will train a very simple model that will compare two numbers and tell us if one is larger than the other.

We'll convert the model into something that can be used by tflite and then run it on the ESP32

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import Input
from tensorflow.data import Dataset
import numpy as np
import itertools
from tensorflow.keras import backend as K

2022-04-22 11:33:02.882823: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-22 11:33:02.882869: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Create a dataset to train our model
We'll create a python generator and feed that through a tensorflow Dataset to train our model

In [2]:
def data_generator():
    while(True):
        number1 = np.random.uniform();
        number2 = np.random.uniform();
        # our input data is an array containing 2 numbers
        X = [number1, number2]
        # our label is 1 or 0
        Y = 1 if number2 > number1 else 0
        # our generator should return the input data and the label
        yield X, [Y]
        
# create a dataset from our generator
train_dataset = tf.data.Dataset.from_generator(
    data_generator, 
    output_types = (tf.float32, tf.int32),
    output_shapes=((2), (1))
)
train_dataset = train_dataset.batch(batch_size=30)

2022-04-22 11:33:11.049482: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-22 11:33:11.049550: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-22 11:33:11.049587: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (vmtg-linux): /proc/driver/nvidia/version does not exist
2022-04-22 11:33:11.050189: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Our very simple mode

We don't need a very complicated model for our problem, so we'll just define a small neural network with an input layer and an output layer.

It's important that the activation function for the output should be sigmoid. This activation function will output a value between 0 and 1.

In [3]:
model = Sequential([
    Input(shape=(2), name= "input_layer" #Shape defines the length of the input vector 
    ),
    Dense(5, activation='relu',name= "hidden_layer"),
    Dense(1, activation='sigmoid',name= "output_layer")
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden_layer (Dense)        (None, 5)                 15        
                                                                 
 output_layer (Dense)        (None, 1)                 6         
                                                                 
Total params: 21
Trainable params: 21
Non-trainable params: 0
_________________________________________________________________


## Compile our model¶
For our loss function we need to use BinaryCrossentropy.

Crossentropy quantifies the difference between two probability distribution.

We have a binary distribution (True or False) so we use binary crossentropy to compare the output from our model with the true distribution.


In [4]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

The first layer has 15 parameters: 
Each neuron has values from the vector (2 values * 5 Neurons = 10) and the biases 

In [5]:
model.fit(
    train_dataset,
    steps_per_epoch=1000,
    epochs=4
)

Epoch 1/4
1000/1000 [==============================] - 12s 11ms/step - loss: 0.4719 - accuracy: 0.8555
Epoch 2/4
1000/1000 [==============================] - 11s 11ms/step - loss: 0.2042 - accuracy: 0.9906
Epoch 3/4
1000/1000 [==============================] - 11s 11ms/step - loss: 0.1269 - accuracy: 0.9948
Epoch 4/4
1000/1000 [==============================] - 11s 11ms/step - loss: 0.0974 - accuracy: 0.9957


## Testing our model
We can feed in some values and see what our model predicts

In [9]:
test_X = np.array([
    [0.1, 0.0],
    [0.3, 0.1],
    [0.5, 0.1],
    [0.7, 0.2]
])
Y = model.predict_on_batch(test_X)
np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})
print(Y)

[[0.17]
 [0.03]
 [0.01]
 [0.00]]


In [11]:
model.weights

[<tf.Variable 'hidden_layer/kernel:0' shape=(2, 5) dtype=float32, numpy=
 array([[3.20, -0.79, -1.83, -1.69, -0.85],
        [-0.86, 1.83, 1.91, 1.86, 1.92]], dtype=float32)>,
 <tf.Variable 'hidden_layer/bias:0' shape=(5,) dtype=float32, numpy=array([1.12, 0.04, 0.44, 0.38, 0.04], dtype=float32)>,
 <tf.Variable 'output_layer/kernel:0' shape=(5, 1) dtype=float32, numpy=
 array([[-1.61],
        [1.67],
        [2.75],
        [3.31],
        [1.43]], dtype=float32)>,
 <tf.Variable 'output_layer/bias:0' shape=(1,) dtype=float32, numpy=array([-0.68], dtype=float32)>]

## Getting the output of the hidden layer

In [14]:
layer_index = 0
output_function = K.function([model.layers[0].input], model.get_layer(index=layer_index).output)

In [15]:
layer_output = output_function([test_X])  # input_data is a numpy array
print(layer_output)

[[1.44 0.00 0.25 0.21 0.00]
 [1.99 0.00 0.08 0.05 0.00]
 [2.64 0.00 0.00 0.00 0.00]
 [3.19 0.00 0.00 0.00 0.00]]


## Saving the model weights

In [10]:
model.save("two_numbers_model.h5")